In [ ]:
# %%
import ee
import geemap
import geopandas as gpd
import json


# Authenticate once (browser popup). After this, ee.Initialize() works.
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

print("✅ Earth Engine initialized")


✅ Earth Engine initialized


In [6]:
# %%
from pathlib import Path
import geopandas as gpd

AOI_GEOJSON = r"C:\GIS\my_water_projects\openwater-shrinking-lake-monitor\data\external\aoi.geojson"

gdf = gpd.read_file(AOI_GEOJSON)
print("CRS:", gdf.crs)
print("Rows:", len(gdf))
print("Geometry types:", gdf.geom_type.value_counts().to_dict())
print("Valid geometries:", gdf.is_valid.value_counts().to_dict())

# Repair: make valid + dissolve to one geometry
# (buffer(0) is a classic fix; make_valid is better if available)
gdf = gdf.to_crs("EPSG:4326")

if hasattr(gdf.geometry, "make_valid"):
    gdf["geometry"] = gdf.geometry.make_valid()
else:
    # Shapely 1.x fallback
    gdf["geometry"] = gdf.geometry.buffer(0)

# Dissolve into one AOI (Earth Engine likes one polygon)
aoi_geom = gdf.union_all()

# Wrap back into a GeoDataFrame
gdf_fixed = gpd.GeoDataFrame(geometry=[aoi_geom], crs="EPSG:4326")

print("After fix - valid:", gdf_fixed.is_valid.values[0])
print("After fix - type:", gdf_fixed.geom_type.values[0])


CRS: EPSG:4326
Rows: 1
Geometry types: {'MultiPolygon': 1}
Valid geometries: {True: 1}
After fix - valid: True
After fix - type: Polygon


In [ ]:
# %%

# Convert fixed AOI geometry to GeoJSON dict
geom_json = gdf_fixed.geometry.iloc[0].__geo_interface__

# Force to pure JSON types (tuples -> lists)
geom_json = json.loads(json.dumps(geom_json))

# Create EE geometry directly
aoi = ee.Geometry(geom_json)

print("✅ AOI converted to ee.Geometry")
print("AOI type:", geom_json["type"])



EEException: Invalid GeoJSON geometry.

In [ ]:
# %%
def mask_s2_sr_clouds(img: ee.Image) -> ee.Image:
    """
    Basic cloud mask for Sentinel-2 Surface Reflectance (S2_SR_HARMONIZED) using QA60 bits.
    Bit 10 = clouds, Bit 11 = cirrus.

    Returns a reflectance-scaled image in 0–1 (divide by 10000).
    """
    qa = img.select("QA60")
    cloud_bit = 1 << 10
    cirrus_bit = 1 << 11

    mask = qa.bitwiseAnd(cloud_bit).eq(0).And(qa.bitwiseAnd(cirrus_bit).eq(0))
    return img.updateMask(mask).divide(10000)


def rgb_monthly_composite(year: int, month: int, aoi: ee.Geometry) -> ee.Image:
    """
    Build a median RGB composite for a given (year, month) over the AOI.
    Uses lenient CLOUDY_PIXEL_PERCENTAGE filtering, then a QA60 mask.
    """
    start = ee.Date.fromYMD(year, month, 1)
    end = start.advance(1, "month")

    col = (
        ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
        .filterBounds(aoi)
        .filterDate(start, end)
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 80))  # broad filter
        .map(mask_s2_sr_clouds)
    )

    # Median composite reduces residual cloud noise and fills coverage gaps across the month
    comp = col.median().clip(aoi)

    return comp


In [ ]:
# %%
# Quick visual sanity check in an interactive map
Map = geemap.Map()
Map.centerObject(aoi, 11)

img_2019 = rgb_monthly_composite(2019, 9, aoi)
img_2025 = rgb_monthly_composite(2025, 9, aoi)

vis = {"min": 0.02, "max": 0.30, "bands": ["B4", "B3", "B2"]}

Map.addLayer(img_2019, vis, "2019-09 RGB (median)")
Map.addLayer(img_2025, vis, "2025-09 RGB (median)")
Map.addLayer(aoi, {}, "AOI outline")
Map


In [ ]:
# %%
# Export to Google Drive as GeoTIFFs
# These become tasks you can monitor in the notebook output or in the EE Tasks UI.

def export_rgb_to_drive(img: ee.Image, aoi: ee.Geometry, name: str, folder: str = "openwater_rgb") -> None:
    geemap.ee_export_image_to_drive(
        image=img.select(["B4", "B3", "B2"]),
        description=name,
        folder=folder,
        fileNamePrefix=name,
        region=aoi,
        scale=10,
        maxPixels=1e13,
    )

export_rgb_to_drive(img_2019, aoi, "s2_rgb_2019_09")
export_rgb_to_drive(img_2025, aoi, "s2_rgb_2025_09")

print("✅ Drive export tasks created.")
print("   Google Drive → My Drive → openwater_rgb/")
